In [8]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-269455
Azure region: eastus2
Subscription id: 9b72f9e6-56c5-4c16-991b-19c652994860
Resource group: aml-quickstarts-269455


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException

cluster_name = "training-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
except ComputeTargetException as e:
    print(e)
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D3_V2", max_nodes=6)
    compute_target = ComputeTarget.create(ws,cluster_name,compute_config)

compute_target.wait_for_completion(show_output=True)

ComputeTargetException:
	Message: ComputeTargetNotFound: Compute Target with name training-cluster not found in provided workspace
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "ComputeTargetNotFound: Compute Target with name training-cluster not found in provided workspace"
    }
}
InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [12]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--C' : uniform(0.5, 1.5),
    '--max_iter': choice(50,100,150)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2,slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yaml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory=".",
    script="train.py",
    arguments=None,
    compute_target=compute_target,
    environment=sklearn_env
)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=src,
    policy=policy,
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    primary_metric_name="Accuracy",
    max_total_runs=20,
    max_concurrent_runs=4,
    hyperparameter_sampling=ps
)

In [17]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)

RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [18]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_8854ad7c-a159-411b-be25-b7c5666ab888
Web View: https://ml.azure.com/runs/HD_8854ad7c-a159-411b-be25-b7c5666ab888?wsid=/subscriptions/9b72f9e6-56c5-4c16-991b-19c652994860/resourcegroups/aml-quickstarts-269455/workspaces/quick-starts-ws-269455&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2024-11-03T19:04:12.6430696Z][GENERATOR][DEBUG]Sampled 4 jobs from search space 
[2024-11-03T19:04:12.8628919Z][SCHEDULER][INFO]Scheduling job, id='HD_8854ad7c-a159-411b-be25-b7c5666ab888_0' 
[2024-11-03T19:04:12.9914738Z][SCHEDULER][INFO]Scheduling job, id='HD_8854ad7c-a159-411b-be25-b7c5666ab888_3' 
[2024-11-03T19:04:12.9903373Z][SCHEDULER][INFO]Scheduling job, id='HD_8854ad7c-a159-411b-be25-b7c5666ab888_2' 
[2024-11-03T19:04:12.9891892Z][SCHEDULER][INFO]Scheduling job, id='HD_8854ad7c-a159-411b-be25-b7c5666ab888_1' 
[2024-11-03T19:04:13.2385628Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_8854ad7c-a159-411b-be25-b7c5666ab888_1' 
[2024-11-03T19:0

{'runId': 'HD_8854ad7c-a159-411b-be25-b7c5666ab888',
 'target': 'training-cluster',
 'status': 'Completed',
 'startTimeUtc': '2024-11-03T19:04:10.951836Z',
 'endTimeUtc': '2024-11-03T19:18:18.184914Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'a4398ba3-2708-402f-9076-e38a4d886312',
  'user_agent': 'python/3.10.11 (Linux-5.15.0-1073-azure-x86_64-with-glibc2.31) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.57.0',
  'best_child_run_id': 'HD_8854ad7c-a159-411b-be25-b7c5666ab888_10',
  'score': '0.9062879339645544',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_8854ad7c-a159-411b-be25-b7c5666ab888_10'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azurem

In [20]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
hpo_metrics = best_run.get_metrics()
print(hpo_metrics)

{'Regularization Strength:': 1.2530588079347846, 'Max iterations:': 100, 'Accuracy': 0.9062879339645545}


## AutoML


In [22]:
from azureml.data.dataset_factory import TabularDatasetFactory

csv_pth = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
ds = TabularDatasetFactory.from_delimited_files(path=csv_pth)

In [23]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd
from azureml.core import Dataset

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

train_df = pd.concat([x_train,y_train], axis=1)

train_df.to_csv('train.csv', index=False)

# set datastore
datastore = ws.get_default_datastore()

# register dataset from pandas dataframe
dataset = Dataset.Tabular.register_pandas_dataframe(
    dataframe=train_df, 
    target=(datastore, 'train.csv'),  
    name='bankmarketing_train_automl' 
)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}
Validating arguments.
Arguments validated.
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'emp.var.rate' -> 'emp_var_rate'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.price.idx' -> 'cons_price_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.conf.idx' -> 'cons_conf_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'nr.employed' -> 'nr_employed'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'job_admin.' -> 'job_admin_'
Column header contains '.' This period will be translated to '_' as we write the data

## Setup and run AutoML


In [24]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=dataset,
    label_column_name='y',
    n_cross_validations=5,
    compute_target=compute_target)

In [25]:
# Submit your automl run

automl_run = exp.submit(automl_config, show_output=True)


Submitting remote run.
No run_configuration provided, running on training-cluster with default configuration
Running on remote compute: training-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_d07e7a33-13df-485a-beea-7bc156bfabf1,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+-------------------------------------

## Save model and Metrics for AutoML


In [ ]:
# Retrieve and save your best automl model.

import joblib

### YOUR CODE HERE ###
best, model = automl_run.get_output()
joblib.dump(model, './training/bestAutoMl.joblib')

In [ ]:
# show metrics
metrics = best.get_metrics()
print(metrics['accuracy'])

In [ ]:
for x in metrics.keys():
    print(f"{x}: {metrics[x]:.3}")